In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import sem
from scipy.integrate import quad
from scipy.stats import gaussian_kde
tqdm.pandas()
import dask
from lmfit import Parameter

%matplotlib widget

## Load and analyze data

In [2]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

In [3]:
data, peak_df = ra.update_data(path, csv_path="peak_df.csv")
variable = data.ryd_data.variables.columns[0]

OSError: no files to open

In [ ]:
calibration_df = pd.read_csv(r"Z:\Projekte - Projects\2020_Aging\2021_06_08\05_sfi_calibration\Analysis\peak_df.csv", index_col=[0, 1, 2])

## Analysis

Extract peaks and sort with respect ro calibration measurement

In [5]:
peak_df_only_p = calibration_df.xs(2, level="InitON").peak_time
peak_df_only_s = calibration_df.xs(0, level="InitON").peak_time

magn_calculator = ra.MagnCalculator(peak_df_only_p, peak_df_only_s, rydberg_states=["56P", "55S"])

In [6]:
fig, ax = plt.subplots()

magn_calculator.plot_calibration(ax=ax)
fig.savefig("calibration.png", bbox_inches="tight", dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
magn = peak_df.peak_time.groupby([variable, "tmstp"]).progress_apply(magn_calculator.maximum_likelyhood)
# magn = magn.unstack()

  0%|          | 0/56 [00:00<?, ?it/s]

In [14]:
magn_summary = pd.concat([magn.groupby(variable).mean(), magn.groupby(variable).sem()], axis=1)
magn_summary.columns = ["magn", "magn_sem"]

In [15]:
fig, ax = plt.subplots()

magn_summary.magn.plot(ax=ax, yerr=magn_summary.magn_sem)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='tmw'>

In [24]:
cosine_model = ra.DampedCosineModel()
cosine_params = cosine_model.guess(magn_summary.magn.values, x=magn_summary.index)
cosine_params["amp"].min = 0
cosine_params["phase"] = Parameter("phase", value=0, vary=False)
if magn_summary.magn_sem.hasnans:
    weights = None
else:
    weights = 1/magn_summary.magn_sem
fitres = cosine_model.fit(
    magn_summary.magn,
    params=cosine_params,
    x=magn_summary.index,
    weights=weights
)

fig, ax = plt.subplots()
fitres.plot_fit(ax=ax, numpoints=100, show_init=False)
times = np.linspace(0, 0.2, 110)
plt.text(
    0.15, 0.95,
    'Omega: %.5g  $\pm$ %.5g gamma: %.5g  $\pm$ %.5g'%(fitres.params['freq'].value, fitres.params['freq'].stderr, fitres.params['gamma'].value, fitres.params['gamma'].stderr),
    transform=ax.transAxes)
ax.set_xlabel(variable + " [us]")
ax.set_ylabel("ion count")

plt.savefig('48P_%s_Fit_Cosine.png'%variable, dpi=600) # add leading zero to the figure name

print("chi^2 reduced", fitres.redchi)
fitres

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

chi^2 reduced 0.03838607550008236


## Analyze ground state density

In [20]:
images = data.ryd_data.images
images.load()

mean_light_image = images.mean("shot")

polygon_mask_indices = [(-500, -80), (500, -80), (500, 90), (-500, 90)]
square_mask = mean_light_image.polygon_mask.get_mask(polygon_mask_indices)
atom_mask = images.elliptical_mask.get_mask(*[25, 11, 200, 200])

background = images.image_05.mean("shot")
absorption = images.image_02
light = images.image_04

atom_images = absorption.where(square_mask) - background
pca = light.pca(n_components=2)
no_atoms_mask = np.logical_not(atom_mask)
reference_images = pca.find_references(absorption, no_atoms_mask) - background

imaging_pca = ra.AbsorptionImaging(atom_images, reference_images, t_exp=150e-3, binning=2)

In [21]:
fit_results = []
for tmstp, image in tqdm(imaging_pca.density.groupby("shot")):
    i = ra.Image(image)
    fit_results.append(i.fitgaussian)
fit_results = pd.concat(fit_results, axis=1).T
fit_results.index = imaging_pca.density.shot.to_index()

  0%|          | 0/706 [00:00<?, ?it/s]

In [22]:
fig, axes = plt.subplots(3, figsize=(6, 5), sharex=True)

ax = axes[0]
fit_results[["center_x", "center_y"]].plot(ax=ax)
ax.set_ylabel("position [$\mu$m]")

ax = axes[1]
fit_results[["sig_x", "sig_y"]].plot(ax=ax)
ax.set_ylabel("position [$\mu$m]")

ax = axes[2]
fit_results["density3d"].plot(ax=ax)
ax.set_ylim(bottom=0)
ax.set_ylabel("density [1/cm³]")

print(fit_results["density3d"].std() / fit_results["density3d"].mean())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.13641005472005635


 Calculate magnetization for a single trace